# **Bigquery Q&A using langchain & LLM (from go/ask-bigquery by nikhilrana@)**


Demo: This notebook guides you on use of LLM to answer questions over a BQ table.
This notebook has also been tested on big datasets(~2 million rows) and works well with low latency.

Dataset: Fitbit public dataset from Kaggle (https://www.kaggle.com/datasets/arashnic/fitbit)

To use it on your own data, change the parameters in the section "App parameters initialization"



Example input: What is the average number of steps taken by the user?

| Date     | Change Log  |
|----------|-------------|
|5/17/2023 |Added Gradio UI at the end |
|5/23/2023 |Changed setup to latest SDK|
|5/30/2023 |Modified Prompt template for metadata queries|
|5/31/2023 |Successfully tested on multiple SQL joins|
|5/31/2023 |Modified prompt template for casting data types for calculations|
|3/07/2023 |Changed BI DB to Clickhouse (jcatalu@, juliahernandez@) |
|6/07/2023 |Full integration with LangChain (jcatalu@, juliahernandez@) |
|6/07/2023 |Updated SDKs to latest Langchain & SQLAlchemy (jcatalu@, juliahernandez@)|
4/09/2023 |Updated SQLDatabasechains moved into langchain experimental (juliahernandez@)|
16/11/2023 |Fixed library dependencies (juliahernandez@)|



### Install latest Vertex LLM SDK and Python libraries setup

In [4]:
# Authenticate with Google account
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip install google-cloud-aiplatform --upgrade


### Python libraries setup
# Install Python Libraries
!pip install langchain
!pip install google-cloud-core
!pip install gradio==3.48.0

# Below libraries are required to build a SQL engine
!pip install SQLAlchemy==1.4.50
!pip install sqlalchemy-bigquery

#Fix dependeny issues
!pip install pydantic==1.10.8

#SQLDatabasechains have been moved into langchain experimental
!pip install -U langchain langchain_experimental

In [ ]:
# Load necessary libraries
import os
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## **Since the kernel restarted at this point, run the below code individually**

### LLM Model Initialization & App parameters initialization

In [5]:
# @title Specify Project details and location of the BQ table

project_id = "qwiklabs-gcp-02-692ce77359c7"  # @param {type:"string"}
location = "us-central1"  # @param {type:"string"}
dataset_id = 'fitbit_data' # @param {type:"string"}
table_name1 = 'user_activity' # @param {type:"string"}
table_name2 = 'user_location' # @param {type:"string"}
table_name3 = 'user_weight_final' # @param {type:"string"}

table_names = (table_name1,table_name2,table_name3)


In [6]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")

# Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset qwiklabs-gcp-02-692ce77359c7.fitbit_data


In [7]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# job configuration
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.PARQUET,
)
# load table_name1
uri = "gs://jrdetorre-fitbit-workshop/user_activity.parquet"
table_ref=f"{project_id}.{dataset_id}.{table_name1}"
load_job = client.load_table_from_uri(
    uri, table_ref, job_config=job_config
)  # Make an API request.
load_job.result()  # Waits for the job to complete.
destination_table = client.get_table(table_ref)
print("{}: Loaded {} rows.".format(table_name1,destination_table.num_rows))

# load table_name2
uri = "gs://jrdetorre-fitbit-workshop/user_location.parquet"
table_ref=f"{project_id}.{dataset_id}.{table_name2}"
load_job = client.load_table_from_uri(
    uri, table_ref, job_config=job_config
)  # Make an API request.
load_job.result()  # Waits for the job to complete.
destination_table = client.get_table(table_ref)
print("{}: Loaded {} rows.".format(table_name2,destination_table.num_rows))

# load table_name3
uri = "gs://jrdetorre-fitbit-workshop/user_weight_final.parquet"
table_ref=f"{project_id}.{dataset_id}.{table_name3}"
load_job = client.load_table_from_uri(
    uri, table_ref, job_config=job_config
)  # Make an API request.
load_job.result()  # Waits for the job to complete.
destination_table = client.get_table(table_ref)
print("{}: Loaded {} rows.".format(table_name3,destination_table.num_rows))




user_activity: Loaded 940 rows.
user_location: Loaded 10 rows.
user_weight_final: Loaded 11 rows.


#### Vertex AI LLM Langchain integration

In [8]:
# Langchain
from langchain.llms import VertexAI

#Initialize Vertex
import vertexai
vertexai.init(project=project_id, location=location)

# LLM model
llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=256,
    temperature=0,
    top_p=0.8,top_k=40,
    verbose=True,
)

## BQ integration

### Create SQL engine for BigQuery

In [9]:
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd

In [14]:
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(table_uri)

In [15]:
query=f"""SELECT * FROM `{project_id}.{dataset_id}.{table_name1}`"""
engine.execute(query).first()

(8053475328, datetime.date(2016, 4, 30), 7135, 5.59000015258789, 5.59000015258789, 0.0, 2.99000000953674, 0.0599999986588955, 2.53999996185303, 0.0, 27, 1, 131, 1281, 2408)

In [16]:
query=f"""SELECT * FROM `{project_id}.{dataset_id}.{table_name2}`"""
engine.execute(query).first()

(7086361926, 'Leeds', 'UK')

In [17]:
query=f"""SELECT * FROM `{project_id}.{dataset_id}.{table_name3}`"""
engine.execute(query).first()

('27.25', datetime.date(2016, 4, 18), None, 4388161847, True, 1461023999000, 69.6999969482422, 153.662190014971)

### SQL Chain setup for LLM

In [18]:
from langchain import SQLDatabase#, SQLDatabaseChain
from langchain_experimental.sql import SQLDatabaseChain #SQLDatabasechains have been moved into langchain experimental
from langchain.prompts.prompt import PromptTemplate

def bq_qna(question):
  #create SQLDatabase instance from BQ engine
  db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=[x for x in table_names])

  #create SQL DB Chain with the initialized LLM and above SQLDB instance
  db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)

  #Define prompt for BigQuery SQL
  _googlesql_prompt = """You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer here"
  Only use the following tables:
  {table_info}

  If someone asks for aggregation on a STRING data type column, then CAST column as NUMERIC before you do the aggregation.

  If someone asks for specific month, use ActivityDate between current month's start date and current month's end date

  If someone asks for column names in the table, use the following format:
  SELECT column_name
  FROM `{project_id}.{dataset_id}`.INFORMATION_SCHEMA.COLUMNS
  WHERE table_name in {table_info}

  Question: {input}"""

  GOOGLESQL_PROMPT = PromptTemplate(
      input_variables=["input", "table_info", "top_k", "project_id", "dataset_id"],
      template=_googlesql_prompt,
  )

  #passing question to the prompt template
  final_prompt = GOOGLESQL_PROMPT.format(input=question, project_id =project_id, dataset_id=dataset_id, table_info=table_names, top_k=10000)

  #pass final prompt to SQL Chain
  output = db_chain(final_prompt)


  return output['result'], output['intermediate_steps'][1]


### Testing the setup

In [19]:
#Testing 1
bq_qna('what is the minimum number of TotalSteps for Id 1644430081')



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Fin

('1223',
 'SELECT MIN(TotalSteps) FROM `qwiklabs-gcp-02-692ce77359c7.fitbit_data.user_activity` WHERE Id = 1644430081')

In [20]:
#Testing 2
bq_qna('what is the highest number of Calories burnt in the month of May by 1644430081')



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Fin

('3404',
 "SELECT MAX(Calories) FROM `qwiklabs-gcp-02-692ce77359c7.fitbit_data.user_activity` WHERE ActivityDate BETWEEN '2016-05-01' AND '2016-05-31' AND Id = 1644430081")

In [ ]:
#Testing 3 - Metadata queries
bq_qna('what are the columns in the table user_activity')

In [22]:
#Testing 4 - Joins
bq_qna('what is the average weight of users living in Mumbai')



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Fin

('52.5999984741211',
 "SELECT AVG(WeightKg) FROM `qwiklabs-gcp-02-692ce77359c7.fitbit_data.user_weight_final` AS t1 JOIN `qwiklabs-gcp-02-692ce77359c7.fitbit_data.user_location` AS t2 ON t1.Id = t2.Id WHERE t2.City = 'Mumbai'")

In [ ]:
#Testing 5 - Joins
bq_qna("""give me lowest steps and weight for user living in Mumbai. I also want their ID""")

In [ ]:
#Testing 6 - Casting data type
bq_qna("""whats the average BMI of users living in India""")

### UI for Demo

In [21]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown(
    """
    ## Ask BigQuery

    This demo is to showcase answering questions on a tabular data available in Big Query using Vertex PALM LLM & Langchain.

    This demo uses a sample public dataset from Kaggle (https://www.kaggle.com/datasets/arashnic/fitbit)

    ### Sample Inputs:
    1. what is the minimum number of steps taken by user 1644430081 ?
    2. what is the average steps covered by 1644430081?
    3. what is the highest number of Calories burnt in the month of May by 1644430081 ?

    ### Enter a search query...

    """)
    with gr.Row():
      with gr.Column():
        input_text = gr.Textbox(label="Question", placeholder="what is the minimum steps taken by 1644430081")

    with gr.Row():
      generate = gr.Button("Ask BigQuery")

    with gr.Row():
      label2 = gr.Textbox(label="Output")
    with gr.Row():
      label3 = gr.Textbox(label="SQL query generated by LLM")

    generate.click(bq_qna,input_text, [label2, label3])
demo.launch(share=True, debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://55a006b67ba84e2f4c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
